# Prudential life insurance      
데이터셋 - Kaggle      
형태 - 파일데이터(csv)       
다운로드 - https://www.kaggle.com/c/prudential-life-insurance-assessment      

# 1. Introduction

 보험은 아날로그 형태의 전통적인 사람중심 규제산업이지만, 신종감염병 COVID-19 확산으로 인하여 언택트, 디지털화가 가속화 되고 있다. 가속화된 디지털화는 디지털 네이티브 세대 뿐만 아니라 40~50대까지 확장되고 있으며 기존의 가치사슬을 허물고 있다. 

 현행 청약 프로세스는 청약일로부터 30일이내 건강검진, 고객방문, 의적위험평가를 위한 서류심사를 포함하여 광범위한 정보를 바탕으로 한 위험평가가 이루어진다. 위험 세분화 및 프로세스 자동화를 통하여 신규 및 기존고객이 보다 신속하고 비용 효과적으로 보험을 가입할 수 있는 것을 목표로 한다. 또한 스코어링 시스템의 예측능력 향상을 통해 효과적인 위험관리를 가능하게 할 것이다.  

 자동화 프로세스 및 스코어링 시스템의 예측력 향상을 통하여 예측적 언더라이팅 리스크 관리를 통하여 위험률차익 개선이 가능하도록 해야한다. 이는 결과적으로 보험료 상승을 막고 공평한 보험료 부담이 가능하게 된다. 추후 고객맞춤형 언더라이팅 프로세스 향상, 자동화를 통하여 고객중심의 고개만족 향상을 위한 지속적인 연구와 개발이 필요할 것이다. 
 
 한편 생명보험산업에 직면하고 있는 문제점은 다음과 같다. IFRS17 대비하여 보장성 보험 중심 포트폴리오로 전환이 필요한 상황이지만 이미 가구당 보험가입률은 98% 초과하는 포화시장 상태이다. 이러한 경쟁상황에서 시장 점유율 향상을 위해 공격적인 상품개발과 언더라이팅 완화로 인하여 결국 상품리스크와 언더라이팅리스크 관리는 매우 어려운 현실이며 결국 보험사기, 보험리스크 증가로 인한 보험료 상승으로 이어질 수 밖에 없는 현실이다. 이러한 문제점 해결하기 위한 방안 또한 함께 고려되어야 할 것이다. 
 


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings


warnings.filterwarnings('ignore')
plt.rc('font', family='NanumBarunGothic') 
plt.rcParams['figure.figsize'] = (10, 7)

pd.set_option('display.float_format', lambda x: '%.2f' % x)

%matplotlib inline 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
filename = '/content/sample_data/train.csv'
train = pd.read_csv(filename)

In [8]:
train.head()

,Id,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_5,Insurance_History_7,Insurance_History_8,Insurance_History_9,Family_Hist_1,Family_Hist_2,Family_Hist_3,Family_Hist_4,Family_Hist_5,Medical_History_1,Medical_History_2,...,Medical_Keyword_10,Medical_Keyword_11,Medical_Keyword_12,Medical_Keyword_13,Medical_Keyword_14,Medical_Keyword_15,Medical_Keyword_16,Medical_Keyword_17,Medical_Keyword_18,Medical_Keyword_19,Medical_Keyword_20,Medical_Keyword_21,Medical_Keyword_22,Medical_Keyword_23,Medical_Keyword_24,Medical_Keyword_25,Medical_Keyword_26,Medical_Keyword_27,Medical_Keyword_28,Medical_Keyword_29,Medical_Keyword_30,Medical_Keyword_31,Medical_Keyword_32,Medical_Keyword_33,Medical_Keyword_34,Medical_Keyword_35,Medical_Keyword_36,Medical_Keyword_37,Medical_Keyword_38,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response
0,2,1,D3,10,0.08,2,1,1,0.64,0.58,0.15,0.32,0.03,12,1,0.00,3,nan,1,2,6,3,1,2,1,1,1,3,1,0.00,1,1,2,2,nan,0.60,nan,0.53,4.00,112,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.00
1,5,1,A1,26,0.08,2,3,1,0.06,0.60,0.13,0.27,0.00,1,3,0.00,2,0.00,1,2,6,3,1,2,1,2,1,3,1,0.00,1,3,2,2,0.19,nan,0.08,nan,5.00,412,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00
2,6,1,E1,26,0.08,2,3,1,0.03,0.75,0.29,0.43,0.03,9,1,0.00,2,0.03,1,2,8,3,1,1,1,2,1,1,3,nan,3,2,3,3,0.30,nan,0.23,nan,10.00,3,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.00
3,7,1,D4,10,0.49,2,3,1,0.16,0.67,0.21,0.35,0.04,9,1,0.00,3,0.20,2,2,8,3,1,2,1,2,1,1,3,nan,3,2,3,3,0.42,nan,0.35,nan,0.00,350,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.00
4,8,1,D2,26,0.23,2,3,1,0.42,0.65,0.23,0.42,0.03,9,1,0.00,2,0.05,1,2,6,3,1,2,1,2,1,1,3,nan,3,2,3,2,0.46,nan,0.41,nan,nan,162,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.00


#2. Data
 데이터는 Train : Test = 3 : 1 비율로 분리되고 무작위 샘플링으로 이루어진다. Train data는 59,766명, 126개 변수로 구성된 transactional data(low level)이고 , Test data는 19,766명, 126개 변수로 구성된다. 변수의 범주는 다음과 같다. 
  * **Product Information** 보험상품 
  * **Insurance Age** 보험나이 
  * **Height** 키
  * **Weight** 몸무게
  * **BMI** 체질량 지수 
  * **Employment Information** 고용정보 
  * **Insured Information** 보험정보
  * **Insurance History** 보험내역
  * **Family History** 가족력
  * **Medical History** 기왕력
  * **Medical Keyword** 메디컬 키워드 

각 범주에는 class 별 여러 변수가 포함되어 있다. 

In [90]:
train.columns

Index(['Id', 'Product_Info_1', 'Product_Info_2', 'Product_Info_3',
       'Product_Info_4', 'Product_Info_5', 'Product_Info_6', 'Product_Info_7',
       'Ins_Age', 'Ht',
       ...
       'Medical_Keyword_40', 'Medical_Keyword_41', 'Medical_Keyword_42',
       'Medical_Keyword_43', 'Medical_Keyword_44', 'Medical_Keyword_45',
       'Medical_Keyword_46', 'Medical_Keyword_47', 'Medical_Keyword_48',
       'Response'],
      dtype='object', length=128)

#3. Exploratory Data Analysis

In [79]:
df = pd.DataFrame(["Product Information",
"Insurance Age",
"Height",
"Weight", 
"BMI",
"Employment Information",
"Insured Information",
"Insurance History",
"Family History",
"Medical History",
"Medical Keyword"], columns = ['Variable_type'])

df['Continous'] = ['1','1','1','1','1','3','0','1','4','0','0']
df['Categorical'] = ['6','0','0','0','0','3','7','8','1','4','0']
df['Dummy'] = ['0','0','0','0','0','0','0','0','0','0','48']
df['Total'] = ['7','1','1','1','1','6', '7', '9', '5', '4', '48'] # row 합을 값으로 하는 column 만들기
df = df.append({'Variable_type':'Sum','Continous': 13, 'Categorical':66, 'Dummy':48, 'Total':1}, ignore_index=True) 
# cloumns 합을 값으로 하는 row 만들기

1. 변수의 유형        
 Continuous, Categorical(명목형 및 순서형) 변수와 Dummy(이진형/지표형) 변수의 유형을 파악한다. 이과정은 모델링 과정에서 변수 선택 및 알고리즘의 선택에 도움이 된다.

 Continous 변수는 요약 통계량, 막대 그래프 및 밀도분포표를 사용하여 분석한다. 범주형 변수는 변수에 따른 결과를 추적하기 위하여 사건 발생률 차트를 사용하여 분석한다.     
  

In [80]:
df

,Variable_type,Continous,Categorical,Dummy,Total
0,Product Information,1,6,0,7
1,Insurance Age,1,0,0,1
2,Height,1,0,0,1
3,Weight,1,0,0,1
4,BMI,1,0,0,1
5,Employment Information,3,3,0,6
6,Insured Information,0,7,0,7
7,Insurance History,1,8,0,9
8,Family History,4,1,0,5
9,Medical History,0,4,0,4


2. 반응도 히스토그램               
 명목형 변수인 고객의 위험분류 1~8등급에 따른 반응 도표이다. 위험등급이 위험도 규모의 크기 순서인지 여부는 언급되지 않았으나, 응답 변수의 분포를 통해 8등급이 가장 낮은 위험도를 나타내는 것으로 유추할 수 있다. 

3. 요약통계          
기계 학습 알고리즘 변수의 보다 쉬운 통합을 위해 [0, 1] 범위로 정규화한다. 요약 통계량은 기본 데이터 분포를 이해하고, box plot 및 density plot 을 통해 데이터를 시각화하는 데 도움이 된다.

4. 연속 변수 분석        



> Box plot 을 사용하면 특히 특이치와 관련하여 데이터 세트를 시각화할 수 있다.



> Density plot 은 평균, 표준 편차 및 첨도와 같은 통계 지표를 포함하여 분포의 특성을 시각화하는 데 도움이 된다. 또한 반응 변수와 어떤 관계가 있는지 시각적으로 확인할 수 있다. 예를 들면 다음과 같다. 고용형태 6번 변수의 밀도분포표는 반응 변수의 히스토그램과 유사하며, 이는 아마도 이 변수가 반응 변수의 좋은 예측 변수가 될 수 있음을 나타낸다.



5. 결측값 분석    



> 결측값 백분율 차트는 변수가 예측을 위한 데이터 레코드가 충분한지 여부를 평가한다. 이 도표는 각 변수의 결측치의 백분율을 나타낸다.



> 이 차트를 통해 결측값 비율이 높은 변수가 반응 변수 예측에 실제로 도움이 되는지 확인할 수 있다. 각 반응 범주에 대한 결측값 분포는 변수 내에 유지되므로 결측값은 본질적으로 무작위하다.



6. 사고 발생률 차트 
  

7. 상관관계표
데이터 분석과 데이터 처리 후, 모델 개발 단계는 변수 감소일 것이다.

#4. 변수 처리       
 

1. 누락된 데이터 처리         
데이터 처리의 예비 단계로, 결측값 비율이 높은 변수는 제거된다. 제거 임계값은 사용자가 결정하는 것이지만, 이 분석에서의 임계값은 30% 이다.            
 


2. 결측값 처리      
이전 모델링 단계에서 삭제되지 않은 변수의 경우, 더 낮은 백분율의 결측값이 있는 변수에 귀속한다. 귀속에 사용되는 방법론은 나머지 데이터의 중위수를 사용하는 것이다. 이는 일반적으로 사용되는 관행이며 모든 변수에 대한 결측 데이터가 반응 변수 위에 랜덤하게 분포되기 때문에 효율적이다.

3. 주성분 분석


4. Boruta Classification Algorithm     
Boruta는 연관성 있는 모든 특징 선택 wrapper algorithmn 이다. 이 방법은 원래 속성의 중요성과 무작위로 시행된 중요도를 비교하여 모든 관련있는 특징에 대하여 top-down 방식으로 수행된다. 관련성이 높은 특징 순서대로 복사본을 만들고, 관련 없는 특징들을 점진적으로 제거한다. 주성분 분석 이후의 모든 속성은 알고리즘에 의해 중요한 것으로 간주된다.

#5. 방법론

1. Multinominal Logic Model      
 데이터를 두 부분으로 나누고 한 부분으로 모델을 예측하고 다른 부분은 모델 유효성 검사와 표본예측을 위해 사용한다.        







> Train data 와 Test data 의 반응 분포를 살펴본다.



> Train & Test data의 분포 비교         
분포는 두 부분에 대해 동일하므로 두 부분이 유사한 분포를 따른다고 가정한다.

> Performance Metrics       
Multinominal Logic Model의 Kappa score 는 0.3183794 이다.

2. Random Forest        
 데이터를 두 부분으로 나누고 한 부분을 사용하여 모델을 예측하고 다른 부분은 모델 유효성 검사와 표본 예측을 위해 사용한다. 전체 데이터 세트를 예측에 사용할 것이다.        
 

> Train data 와 Test data 의 반응 분포를 살펴본다.



> Train & Test data의 분포 비교         
 분포는 두 부분에 대해 동일하므로 두 부분이 유사한 분포를 따른다고 가정한다.    

> Performance Metrics           
 Random Forest의 Kappa Score는 0.4744071이다.           
Random Forest 예측에 사용되는 Rattel plot(or tree) 은 다음과 같다.

3. XGBoost            
XGBoost는 "Extreme Gradient Boosting"의 줄임말로, 여기서 "Gradient Boosting"이라는 용어는 Greedy function approximation 논문에서 제안했다 : Gradient Boosting machine, by Friedman.       
XGBoost는 이 original model 을 기반으로 한다. XGBoost는 학습 문제를 관라하는 데 자주 사용되는데, training data(with multiple features, xi)를 사용하여 target variable(yi)을 예측하는데 사용된다.